## Домашнее задание №5. «Гибридные рекомендатльные системы»
 - Датасет ml-latest
- Вспомнить подходы, которые мы разбирали
- Выбрать понравившийся подход к гибридным системам
- Написать свою

### 1. Для начала, найдем 10 самых похожих юзеров на данного юзера.
### 2. Возьмем из последних фильмов три фильма, которые эти пользователи оценили больше, чем на 4.5.
### 3. Получим 30 фильмов, для них предскажем оценку и отсортирует по убыванию.
### 4. Пять фильмов с самой большой оценкой будут нашими рекомендациями.

In [1]:
from surprise import SVD, SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from scipy.spatial.distance import jaccard

import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

import pandas as pd
import numpy as np

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [4]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [5]:
num_movies = movies_with_ratings.movieId.unique().shape[0]
uniques = movies_with_ratings.movieId.unique()

In [6]:
user_vector = {}

for user, group in movies_with_ratings.groupby('userId'):
    user_vector[user] = np.zeros(num_movies)
    
    for i in range(len(group.movieId.values)):
        m = np.argwhere(uniques==group.movieId.values[i])[0][0]
        r = group.rating.values[i]
        user_vector[user][m] = r

In [7]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [8]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [9]:
trainset, testset = train_test_split(data, test_size=.15, random_state=42)

In [10]:
algo = SVDpp(n_factors=20, n_epochs=20)
algo.fit(trainset)

In [11]:
test_pred = algo.test(testset)

In [12]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8567


0.8566636539565012

In [13]:
def user_recommendation(uid):

    titles = []
    distances = []

    for key in user_vector.keys():
        if key == uid:
            continue

        titles.append(key)
        distances.append(jaccard(user_vector[uid], user_vector[key]))

    best_indexes = np.argsort(distances)[:10]
    similar_users = np.array([(titles[i], distances[i]) for i in best_indexes])[:, 0]

    movies_with_ratings.sort_values('timestamp', inplace=True)

    movies = np.array([])
    for user in similar_users:
        a = np.array(movies_with_ratings[movies_with_ratings.rating >= 4.5][movies_with_ratings.userId == user][-3:].title)
        movies = np.concatenate([a, movies])

    user_movies = movies_with_ratings[movies_with_ratings.userId == uid].title.unique()

    scores = []
    titles = []

    for movie in movies:
        scores.append(algo.predict(uid=uid, iid=movie).est)
        titles.append(movie)

    best_indexes = np.argsort(scores)[-5:]
    for i in reversed(best_indexes):
        print(titles[i], scores[i])

In [16]:
user_recommendation(1.0)

C:\Users\Arsenii\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Boot, Das (Boat, The) (1981) 5.0
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981) 5.0
Young Frankenstein (1974) 4.810313944136407
Good Morning, Vietnam (1987) 4.787735520531383
Go (1999) 4.708185997291376
